In [1]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=24, split_factor=24)

### get basin boundary files

In [ ]:
basins = glob.glob('../../data/GRIT/full_catchment/GRIT_full_catchment_*_EPSG8857_simplify_final.gpkg')
if not os.path.exists('../basin_boundary'):
    os.mkdir('../basin_boundary')
for basin in basins:
    gdf = read_dataframe(basin)
    
    # difference between ohdb_darea and grit_darea less than 20%
    gdf['bias'] = np.abs(gdf.grit_darea - gdf.ohdb_darea_hydrosheds) / gdf.ohdb_darea_hydrosheds * 100
    gdf1 = gdf.loc[gdf.bias<=20,:]
    
    # darea greater than 125 km2 to ensure at least one grid cell
    gdf1 = gdf1.loc[gdf1.grit_darea>=125,:]

    gdf1['segment_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1['reach_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1 = gdf1.rename(columns={'grit_darea':'gritDarea','ohdb_darea_hydrosheds':'ohdbDarea1','ohdb_darea':'ohdbDarea0'})
    
    # save
    basin1 = os.path.basename(basin)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.gpkg')
    gdf1 = gdf1.to_crs('epsg:4326')
    basin1 = re.sub('EPSG8857','EPSG4326',basin1)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.shp')
    print(basin)

### transform MSWX meteo files to seperate files for each gauge

In [2]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=16, split_factor=16)

# transform all the meteo files into separate csv files for each gauge
def read(year, meteo = 'MSWX'):
    if meteo == 'ERA5':
        fname = f'../ee_era5_land/ERA5_Land_daily_meteorology_for_OHDB_10717_stations_{year}.csv'
    elif meteo == 'MSWX':
        fname = f'../data_mswx/MSWX_daily_meteorology_for_OHDB_10717_stations_{year}.csv'
    df = pd.read_csv(fname).set_index('ohdb_id')
    print(year)
    return df
pool = mp.Pool(8)
df_meteo = pool.map(read, np.arange(1982, 2024).tolist())
df_meteo = pd.concat(df_meteo, axis = 1)
df_meteo = df_meteo.rename(columns=lambda x:x.lower())
df_meteo = df_meteo.round(6)
df_meteo.loc[:,df_meteo.columns.str.endswith(('_p','_tmax','_tmin','_wind'))] = df_meteo.loc[:,df_meteo.columns.str.endswith(('_p','_tmax','_tmin','_wind'))].round(2)
df_meteo = df_meteo.reset_index()

def func_main(x, meteo = 'MSWX'):
    ohdb_id = x.ohdb_id
    if os.path.exists(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv'):
        try:
            a = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            if a.shape[0] < 15300:
                os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
            else:
                return
        except:
            os.remove(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    x = x.drop(index=['ohdb_id'])
    x.name = 'value'
    y = []
    for name in ['p','tmax','tmin','lwd','pres','relhum','spechum','swd','wind']:
        y0 = x.loc[x.index.str.endswith('_'+name)]
        y0.name = name
        y0.index = y0.index.str[:8]
        y.append(y0)
    y = pd.concat(y, axis = 1)
    # x = x.pivot_table(index = 'date', columns = 'meteo', values = 'value').rename(columns=lambda x:x.lower()).reset_index()
    # x['date'] = pd.to_datetime(x.date.values)
    # x.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)
    y['date'] = pd.to_datetime(y.index.values, format = '%Y%m%d')
    y.to_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv', index = False)

df_meteo.p_apply(func_main, axis = 1)

1984
1994
1988
1986
1996
1982
1990
1992
1995
1985
1997
1989
1991
1987
1983
1993
1998
2000
2002
2004
2006
2008
2010
2012
1999
2001
2003
2005
2007
2009
2011
2013
2014
2016
2018
2020
2022
2015
2017
2019
2021
2023


FUNC_MAIN DONE:   0%|          | 0/10717 [00:00<?, ?it/s]

0        None
1        None
2        None
3        None
4        None
         ... 
10712    None
10713    None
10714    None
10715    None
10716    None
Length: 10717, dtype: object

### connect Qmax7 and Qmin7 with meteo

In [3]:
def func_meteo(x, name = 'Qmax7date'):
    ohdb_id = x.xxx.values[0]
    x = x.drop(columns=['xxx'])
    df_meteo = pd.read_csv(f'../data_mswx/mswx_each_basin/{ohdb_id}.csv')
    df_meteo['date'] = pd.to_datetime(df_meteo['date'])
    df_meteo['ohdb_id'] = ohdb_id
    x = x[['ohdb_id',name]].merge(df_meteo,on = 'ohdb_id')
    x['tmp'] = (x.date - x[name]).dt.days
    x3 = x.loc[(x.tmp>-3)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_3')
    x7 = x.loc[(x.tmp>-7)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_7')
    x15 = x.loc[(x.tmp>-15)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_15')
    x30 = x.loc[(x.tmp>-30)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_30')
    x365 = x.loc[(x.tmp>-365)&(x.tmp<=0),:].drop(columns=['date','ohdb_id','tmp']).groupby(name).mean().rename(columns=lambda x:x+'_365')
    x = pd.concat([x3,x7,x15,x30,x365], axis = 1).reset_index()
    return x

df_flood = pd.read_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023_80days.csv')
df_flood['Qmax7date'] = pd.to_datetime(df_flood['Qmax7date'])
df_flood['Qmin7date'] = pd.to_datetime(df_flood['Qmin7date'])

df_flood['xxx'] = df_flood['ohdb_id'].values

df2 = df_flood.groupby('ohdb_id').p_apply(lambda x: func_meteo(x, name = 'Qmax7date')).reset_index().drop(columns = ['level_1'])
df2.to_csv('../data/Qmax7_seasonal4_multi_MSWX_meteo.csv', index = False)
del df2

df2 = df_flood.groupby('ohdb_id').p_apply(lambda x: func_meteo(x, name = 'Qmin7date')).reset_index().drop(columns = ['level_1'])
df2.to_csv('../data/Qmin7_seasonal4_multi_MSWX_meteo.csv', index = False)

<LAMBDA> DONE:   0%|          | 0/10717 [00:00<?, ?it/s]

ohdb_id
OHDB_014011003     70
OHDB_007001160     71
OHDB_014031590     72
OHDB_014029420     73
OHDB_009000638     74
                 ... 
OHDB_014005487    166
OHDB_014005495    166
OHDB_014005518    166
OHDB_014024189    166
OHDB_014000018    166
Name: p_3, Length: 10717, dtype: int64

### select OHDB gauge and calculate streamflow indices

In [5]:
def cleanQ(df):
    # eliminate invalid records
    df1 = df.loc[df.Q.apply(lambda x: not isinstance(x, str)),:]
    df2 = df.loc[df.Q.apply(lambda x: isinstance(x, str)),:]
    try:
        df2 = df2.loc[df2.Q.str.match('\d+'),:]
    except:
        pass
    df = pd.concat([df1, df2])
    df['Q'] = df.Q.astype(np.float32)
    return df

def del_unreliableQ(df):
    '''observations less than 0 were flagged as
        suspected, and (b) observations with more than ten consecutive
        equal values greater than 0 were flagged as suspected'''
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').set_index('date')
    index = pd.date_range(df.index[0], df.index[-1], freq = 'D')
    df = df.reindex(index).fillna(0)
    df1 = df.diff()
    df1 = df1.where(df1==0, 1).diff()
    start = np.where(df1.values==-1)[0]
    end = np.where(df1.values==1)[0]
    if len(start) == 0 or len(end) == 0:
        # must no less than zero
        df = df.loc[df.Q>=0,:]
        return (df)
    if start[0] > end[0]:
        start = np.array([0]+start.tolist())
    if start[-1] > end[-1]:
        end = np.array(end.tolist()+[df1.shape[0]+10])
    duration = end - start
    start = start[duration>=10]
    end = end[duration>=10]
    del_idx = np.array([item for a,b in zip(start,end) for item in np.arange(a+1,b+2).tolist()])
    del_idx = del_idx[del_idx<df.shape[0]]
    if len(del_idx) > 0:
        df.drop(df.index[del_idx], inplace = True)
    # must no less than zero
    df = df.loc[df.Q>=0,:]
    return (df)

def main(par, scale = 'season'):
    ohdb_id, Darea = par
    df = pd.read_csv(os.environ['DATA']+f'/data/OHDB/OHDB_v0.2.3/OHDB_data/discharge/daily/{ohdb_id}.csv')
    # read
    df = cleanQ(df)
    # quality check
    df = del_unreliableQ(df)
#     # only retain records with at least 328 observations (90%) are required
#     tmp = df.resample('Y')['Q'].agg(countDay = lambda x:x.shape[0])
#     if tmp.loc[tmp.countDay>=328,:].shape[0] == 0:
#         return
#     years = tmp.loc[(tmp.countDay>=328)&(tmp.index.year>=1982),:].index.year.tolist()
#     if tmp.loc[(tmp.countDay>=300)&(tmp.index.year==2023),:].shape[0] > 0:
#         years = years + [2023]
    years = np.arange(1982, 2024).tolist()
    df = df.loc[df.index.year.isin(years),:]
#     # only retain gauge with at least 20 years of AMS during 1982-2023
#     if len(years) < 20:
#         return
    # reindex
    newindex = pd.date_range(df.index.values[0], df.index.values[-1], freq = 'D')
    df = df.reindex(newindex)
    # 7-day moving average
    df = df.rolling(7).mean().dropna()
    df['year'] = df.index.year
#     df['season'] = 'DJF'
#     df.loc[(df.index.month>=3)&(df.index.month<=5),'season'] = 'MAM'
#     df.loc[(df.index.month>=6)&(df.index.month<=8),'season'] = 'JJA'
#     df.loc[(df.index.month>=9)&(df.index.month<=11),'season'] = 'SON'
    df['season'] = '11-4'
    df.loc[(df.index.month>=5)&(df.index.month<=10),'season'] = '5-10'
    if scale == 'year':
        # count observations and calculate Qmax7 and Qmin7 for each year
        df1 = df.groupby('year')['Q'].agg(countDay = lambda x:x.shape[0], 
                                        Qmax7 = lambda x:x.max(),
                                        Qmin7 = lambda x:x.min(),
                                        Qmax7date = lambda x:x.idxmax(),
                                        Qmin7date = lambda x:x.idxmin(),
                                        )
    elif scale == 'season':
        # count observations and calculate Qmax7 and Qmin7 for each season
        df1 = df.groupby([scale,'year'])['Q'].agg(countDay = lambda x:x.shape[0], 
                                        Qmax7 = lambda x:x.max(),
                                        Qmin7 = lambda x:x.min(),
                                        Qmax7date = lambda x:x.idxmax(),
                                        Qmin7date = lambda x:x.idxmin(),
                                        )
#         df1 = df1.loc[df1.countDay>=80,:] # at least 80 days of records to calculate seasonal extremes
        df1 = df1.loc[df1.countDay>=150,:] # at least 150 days of records to calculate double-seasonal extremes
    else:
        raise Exception('scale must be season or year')
    df1['Qmax7date'] = pd.to_datetime(df1['Qmax7date'])
    df1['Qmin7date'] = pd.to_datetime(df1['Qmin7date'])
    
    # if scale == 'season':
    #     # keep events independent
    #     thres = 5 + np.log(Darea * 0.386102) # thres for Qmax7
    #     df1_Qmax = df1[['Qmax7','Qmax7date']].sort_values('Qmax7date')
    #     df1_Qmax = df1_Qmax.loc[~(df1_Qmax.Qmax7date.diff().dt.days<thres),:]
    #     thres = 30 # thres for Qmin7
    #     df1_Qmin = df1[['Qmin7','Qmin7date']].sort_values('Qmin7date')
    #     df1_Qmin = df1_Qmin.loc[~(df1_Qmin.Qmin7date.diff().dt.days<thres),:]
    #     df1 = pd.concat([df1_Qmax, df1_Qmin], axis = 1)

    # Qmax7 and Qmin7 must not be lower than zero
    df1['Qmax7'] = df1.Qmax7.where(df1.Qmax7>=0, np.nan)
    df1['Qmin7'] = df1.Qmin7.where(df1.Qmin7>=0, np.nan)

    # # Qmax7 cannot be lower than 50% percentile of daily discharges between 1982-2023
    # q = df.loc[df.Q>0,'Q'].quantile(0.5)
    # df1['Qmax7'] = df1.Qmax7.where(df1.Qmax7 >= q, np.nan)
    # # Qmin7 cannot be greater than 50% percentile of daily discharges between 1982-2023
    # df1['Qmin7'] = df1.Qmin7.where(df1.Qmin7 <= q, np.nan)

    if df1.shape[0] == 0:
        return
    df1 = df1.reset_index()
    df1['ohdb_id'] = ohdb_id
    print(ohdb_id)
    return (df1)

if __name__ == '__main__':
    # if not os.path.exists('../data/OHDB_metadata_subset.csv'):
    #     # select gauges that have good basin boundary
    #     df = pd.read_csv('../../data/OHDB/OHDB_v0.2.3/OHDB_metadata/OHDB_metadata.csv')
    #     df1 = []
    #     for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    #         gdf = read_dataframe(fname, read_geometry = False)
    #         print(gdf.shape, gdf.ohdb_id.unique().shape)
    #         df1.append(df.loc[df.ohdb_id.isin(gdf.ohdb_id.unique()),:])
    #     df1 = pd.concat(df1)
    #     df1.to_csv('../OHDB_metadata_subset.csv', index = False)
    # else:
    df1 = pd.read_csv('../data/basin_attributes.csv')
    print(df1.shape)
    ohdb_ids = df1.ohdb_id.values
    pool = mp.Pool(48)
    pars = df1[['ohdb_id','gritDarea']].values.tolist()
    df = pool.map(main, pars)
    df = pd.concat(df)
    df.to_csv('../data/dis_OHDB_seasonal_Qmin7_Qmax7_1982-2023.csv', index = False)
    print(df.Qmin7.isna().sum(), df.Qmax7.isna().sum())

(10717, 66)
OHDB_002000683
OHDB_007005041OHDB_009000845OHDB_008004429OHDB_012000686OHDB_014011019OHDB_014003918

OHDB_001000705OHDB_011001343


OHDB_002000684OHDB_007004811
OHDB_014039706
OHDB_011001039OHDB_008000953OHDB_014031282OHDB_008001574OHDB_014020473



OHDB_006002577OHDB_012002737


OHDB_009000844OHDB_011001432
OHDB_014030718OHDB_007008093
OHDB_007009733OHDB_011000924OHDB_014018150
OHDB_007008819OHDB_014030677

OHDB_006004656OHDB_007009037OHDB_007009859OHDB_014022562OHDB_011000484OHDB_006003449OHDB_014021493

OHDB_011000604OHDB_007006237


OHDB_001000634
OHDB_006003533OHDB_012001995OHDB_014011858
OHDB_014020484


OHDB_006002623
OHDB_007008845

OHDB_014034284


OHDB_003000053





OHDB_014000054

OHDB_010000050OHDB_007008488OHDB_011000556OHDB_009000779OHDB_014031283

OHDB_011000701OHDB_014021442OHDB_007006490


OHDB_009000271OHDB_014023472OHDB_007009994

OHDB_006002925

OHDB_012002768
OHDB_006002503OHDB_014019391OHDB_014023342OHDB_006003941

OHDB_011000022
OHDB_014012348
OHDB_0

OHDB_012002287OHDB_003000210OHDB_014011063OHDB_003000049
OHDB_006004079OHDB_012002065OHDB_014020496
OHDB_014018516OHDB_014031304

OHDB_009000550
OHDB_011000965OHDB_014017840
OHDB_014022077OHDB_014022115OHDB_011001062

OHDB_011000061OHDB_010000021OHDB_007004962
OHDB_001000474OHDB_014022408OHDB_011001588OHDB_006000525OHDB_011001498OHDB_014012428

OHDB_011000234




OHDB_014018169


OHDB_014021606






OHDB_006003512OHDB_007006176
OHDB_011001287OHDB_006001786OHDB_006002601OHDB_014028345
OHDB_012001523OHDB_007008144OHDB_006004244

OHDB_014017090

OHDB_014032230OHDB_014029775



OHDB_007008229OHDB_003000002OHDB_006002000OHDB_006002403
OHDB_011000515OHDB_007009708OHDB_007009534


OHDB_012000246
OHDB_014018428OHDB_007009657

OHDB_006002084OHDB_014000124






OHDB_001000498
OHDB_014026556OHDB_014011054
OHDB_014017863OHDB_014011663OHDB_001000337OHDB_006002600OHDB_014036411
OHDB_014018563
OHDB_007007770OHDB_014027501
OHDB_010000369OHDB_006001953OHDB_006004713







OHDB_009000387
OHDB_0140111

OHDB_011000383OHDB_009000519OHDB_014022883
OHDB_007009917







OHDB_014027455

OHDB_014012594



OHDB_014021844

OHDB_011001586OHDB_014017806


OHDB_012002209OHDB_007008910OHDB_007004979OHDB_006000912OHDB_014003729

OHDB_014017702

OHDB_006003467

OHDB_014023849OHDB_002000016OHDB_007010240OHDB_008001614
OHDB_014016361OHDB_006003238OHDB_007009719
OHDB_014016996
OHDB_011000983



OHDB_011000265


OHDB_011000865OHDB_006003025OHDB_014010912OHDB_007010241OHDB_010000148

OHDB_006002103
OHDB_006003958OHDB_014011837
OHDB_009000521

OHDB_011001364
OHDB_014012397
OHDB_014030879OHDB_011001067

OHDB_007008817
OHDB_014003743

OHDB_014023864
OHDB_012000046OHDB_011001091

OHDB_014033266
OHDB_007009997OHDB_007006528OHDB_007008564
OHDB_014032354
OHDB_006001067
OHDB_014020543

OHDB_011001027OHDB_006002708

OHDB_014018940

OHDB_011001357




OHDB_006001770
OHDB_012001665OHDB_008000706
OHDB_011001295OHDB_011001589

OHDB_014011018OHDB_014019605OHDB_010000105OHDB_014011651OHDB_014016211OHDB_006004184OHDB_

OHDB_014035765OHDB_014014943OHDB_007000162OHDB_004004943OHDB_014019064



OHDB_014032593



OHDB_014028823OHDB_014003443OHDB_007003749

OHDB_014018740OHDB_014010608OHDB_001000590





OHDB_009000854OHDB_014027174OHDB_014005813
OHDB_014023835
OHDB_014033236

OHDB_007004327OHDB_014011528OHDB_014027253OHDB_014010326OHDB_004001796OHDB_001000103
OHDB_014007687
OHDB_004002149OHDB_004001309OHDB_014006767
OHDB_014001617
OHDB_009000198OHDB_014032887
OHDB_004003227
OHDB_007000242OHDB_014002507OHDB_014026995


OHDB_014008360


OHDB_001000074
OHDB_014000112

OHDB_014027077
OHDB_014001020OHDB_014005418OHDB_004004459
OHDB_014003694


OHDB_009000010
OHDB_014005956OHDB_014019087



OHDB_014018337OHDB_001000710





OHDB_014031635

OHDB_014023836
OHDB_004004945OHDB_001000045
OHDB_014006768

OHDB_014025966OHDB_014002779
OHDB_007004019
OHDB_007007066OHDB_014027262OHDB_014033390OHDB_007000229OHDB_014004284OHDB_001000602OHDB_014001772OHDB_001000599


OHDB_014015160
OHDB_008000119
OHDB_007007165OHDB_0070073

OHDB_004000334OHDB_014009800OHDB_007003856OHDB_004005084
OHDB_004001144
OHDB_007007074
OHDB_004002468OHDB_014003094OHDB_014024261


OHDB_008005658OHDB_001000654
OHDB_014008796
OHDB_014033876
OHDB_014023045OHDB_014023372
OHDB_014014985
OHDB_015000252

OHDB_007004024

OHDB_014003490OHDB_007007097OHDB_007004357OHDB_014035561
OHDB_014004645
OHDB_014000249OHDB_014005314OHDB_009000205
OHDB_014032607
OHDB_014036027OHDB_007005178OHDB_009000001




OHDB_007004117
OHDB_007004027OHDB_009000847




OHDB_008001804OHDB_007004389
OHDB_004001433
OHDB_004002391OHDB_014010413
OHDB_014002747
OHDB_007004171
OHDB_014001218
OHDB_004005015OHDB_007006929OHDB_014023215OHDB_014008601OHDB_014026377
OHDB_014035220

OHDB_006000230

OHDB_014006747


OHDB_014004632
OHDB_007005592
OHDB_014033877OHDB_014030066
OHDB_004001803
OHDB_014005726

OHDB_007003754

OHDB_014015903OHDB_014035840OHDB_009000869



OHDB_008000713OHDB_009000284
OHDB_008000798OHDB_009000498OHDB_014003087OHDB_014005129



OHDB_007003930




OHDB_00400

OHDB_014032788
OHDB_001000345OHDB_007009725OHDB_001000343OHDB_007010157
OHDB_007008907OHDB_006001897

OHDB_006004918OHDB_011000830


OHDB_006004170OHDB_001000501

OHDB_014004073

OHDB_011000933




OHDB_009000629OHDB_014002623
OHDB_011001485
OHDB_007008453OHDB_007008961OHDB_007007801OHDB_011000750OHDB_011000410
OHDB_007005607

OHDB_007009946OHDB_001000481OHDB_007010160OHDB_001000592OHDB_007008079

OHDB_007010032OHDB_007009225
OHDB_001000109


OHDB_007008031



OHDB_014034551OHDB_007003868
OHDB_004001888


OHDB_007003762
OHDB_007009937OHDB_007000076


OHDB_014002504
OHDB_014004064OHDB_007004403



OHDB_007009384OHDB_007010153
OHDB_001000464OHDB_007008767OHDB_001000288

OHDB_007010320OHDB_004002056OHDB_012000799
OHDB_007009572OHDB_007004203OHDB_007009604OHDB_014006810
OHDB_014004489OHDB_007004790

OHDB_007009371
OHDB_007008997OHDB_001000175OHDB_012001923OHDB_007008481
OHDB_001000152OHDB_014034555

OHDB_014032246OHDB_011001082


OHDB_001000179





OHDB_001000496

OHDB_007009284OHDB_00100

OHDB_001000440OHDB_007008314
OHDB_014000018
OHDB_007005783OHDB_001000314



OHDB_008000569

OHDB_007009559
OHDB_007008327OHDB_008004273
OHDB_014003036OHDB_007008839
OHDB_014012139



OHDB_014024737
OHDB_007004545
OHDB_011000569OHDB_014021551
OHDB_006001079OHDB_001000242OHDB_007008404





OHDB_007009891
OHDB_007010354OHDB_002000006
OHDB_007005716OHDB_014015682OHDB_001000269
OHDB_012000859

OHDB_014016815OHDB_001000384OHDB_007010307OHDB_008001228



OHDB_007008443OHDB_007009173OHDB_007006097


OHDB_011001468
OHDB_007009361OHDB_007009393OHDB_007005818OHDB_001000117OHDB_014000070
OHDB_007008856OHDB_007008895OHDB_001000315
OHDB_011000268OHDB_014031107

OHDB_014021738OHDB_007009071OHDB_007004273

OHDB_014003879






OHDB_007008783



OHDB_014019984OHDB_006004254


OHDB_007004197OHDB_007005731OHDB_007009238OHDB_007005653OHDB_008000607OHDB_007010256OHDB_005000376OHDB_007007931OHDB_007005576OHDB_007008485OHDB_007010150

OHDB_007008855OHDB_007008145
OHDB_014021607
OHDB_007008804

OHDB_00700841

OHDB_006003568OHDB_014004602OHDB_014019946OHDB_014033864
OHDB_001000390
OHDB_006001116OHDB_014011402OHDB_001000444
OHDB_006001251
OHDB_014012832OHDB_007009115



OHDB_014010720OHDB_002000196


OHDB_006002814OHDB_014015990OHDB_007004922OHDB_014019885OHDB_012001299




OHDB_001000351

OHDB_012001183OHDB_007008792OHDB_007008480

OHDB_007006190
OHDB_007008726OHDB_006004108


OHDB_007006086OHDB_007008061
OHDB_007005588

OHDB_007009346OHDB_007008470



OHDB_007004957OHDB_014035424



OHDB_011001200OHDB_007004754OHDB_007008154


OHDB_007004742OHDB_012002088OHDB_014017228


OHDB_014021750
OHDB_007004760OHDB_007009185
OHDB_014017808
OHDB_002000208

OHDB_007007800OHDB_007005038
OHDB_007005770


OHDB_006001561OHDB_007009877
OHDB_008002205


OHDB_014021177OHDB_001000386OHDB_007004932
OHDB_006003574OHDB_007008008OHDB_007009362
OHDB_014000004OHDB_007006146
OHDB_007006209



OHDB_014020515



OHDB_014018951OHDB_007009863
OHDB_007004765OHDB_007009403OHDB_014023627OHDB_001000356
OHDB_014023267
OHDB_007

OHDB_007000331

OHDB_007005190
OHDB_007005589

OHDB_004003988

OHDB_007004853OHDB_004004458OHDB_004001924

OHDB_007005386OHDB_008003111OHDB_014016454OHDB_007007183OHDB_008001364
OHDB_004001115
OHDB_004004267
OHDB_014020603OHDB_015000506

OHDB_014010467
OHDB_007005461OHDB_014035423
OHDB_007005315OHDB_007000088


OHDB_004002527


OHDB_014017070


OHDB_007002014
OHDB_014014923
OHDB_004001154OHDB_004001925OHDB_004000481OHDB_014005388





OHDB_007005585
OHDB_004003966OHDB_007006197

OHDB_004001585OHDB_014032374

OHDB_014025930OHDB_014021375OHDB_007004599

OHDB_014010456OHDB_007004604OHDB_007004646


OHDB_004003157OHDB_004002886


OHDB_014019216OHDB_014005536OHDB_007006635


OHDB_007001111OHDB_004001920
OHDB_007006075
OHDB_004001508
OHDB_008003942OHDB_007004739
OHDB_007005357OHDB_004002885OHDB_007006536
OHDB_007007256OHDB_014017896


OHDB_004003182

OHDB_007000369OHDB_007005299OHDB_008003102OHDB_007007248
OHDB_007001160

OHDB_014006086



OHDB_007005603
OHDB_004006165
OHDB_007006572OHDB_004

OHDB_004003519
OHDB_004004843
OHDB_007000419
OHDB_008003611
OHDB_007004608OHDB_007007670



OHDB_014025335OHDB_014029082
OHDB_004001322
OHDB_007004805OHDB_004004438
OHDB_007007022OHDB_004000070
OHDB_004001617

OHDB_014006420OHDB_014026614
OHDB_014004684
OHDB_004002060
OHDB_007004756OHDB_004000971OHDB_004003274
OHDB_004002441OHDB_007006510





OHDB_014009573OHDB_014026415

OHDB_014005212OHDB_007006341


OHDB_004001146
OHDB_004005449OHDB_015000209
OHDB_007000186OHDB_014029219

OHDB_014015344OHDB_004000191
OHDB_007005002OHDB_007004762OHDB_014014896OHDB_004000132OHDB_007005403
OHDB_014003386OHDB_014025920
OHDB_004004822OHDB_014003492

OHDB_007001804


OHDB_015000535OHDB_004000197
OHDB_004002458OHDB_007007693




OHDB_004004452OHDB_014021860

OHDB_007007666
OHDB_004002059OHDB_014006569OHDB_014008928
OHDB_008002272
OHDB_007006319
OHDB_014032627


OHDB_007007072OHDB_007007483OHDB_014019881OHDB_004003533OHDB_014033391OHDB_008004282

OHDB_004003755






OHDB_007005010OHDB_014008505OHDB_004003

### subset basin boundary files again

In [ ]:
df = pd.read_csv('../dis_OHDB_Qmin7_Qmax7_1982-2023.csv')
ohdb_ids = df.ohdb_id.unique()
for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    gdf = read_dataframe(fname)
    gdf = gdf.loc[gdf.ohdb_id.isin(ohdb_ids),:]
    write_dataframe(gdf, fname[:-5]+'_subset.gpkg')
    print(fname)

### use catch_mean_GLHYMPS_GLiM.py to get catchment average subsurface characteristics

### calculate number of upstream dams

In [ ]:
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp')
gdf_dam = gdf_dam.to_crs('espg:8857')
gdf = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')
join = gpd.overlay(gdf_dam, gdf)
join = join.groupby('ohdb_id')['Feature_ID'].count().rename(columns={'Feature_ID':'dam_num'})
join = join.reindex(gdf.ohdb_id.values).fillna(0).reset_index()
join.to_csv('../geography/dam_num.csv', index = False)

### extract average meteorological conditions in the past 7 days preceding Qmax7 and Qmin7

### merge basin attributes

In [ ]:
fnames = glob.glob('../geography/*csv')
df_all = []
for fname in fnames:
    df = pd.read_csv(fname)
    if df.shape[0] == 1:
        df = df.T
    if df.shape[0] == 10717:
        df = df.set_index('ohdb_id')
    name = os.path.basename(fname).split('_')[0]
    if '0-5cm' in fname:
        name = name + '_layer1'
    elif '5-15cm' in fname:
        name = name + '_layer2'
    elif '15-30cm' in fname:
        name = name + '_layer3'
    elif '30-60cm' in fname:
        name = name + '_layer4'
    elif '60-100cm' in fname:
        name = name + '_layer5'
    elif '100-200cm' in fname:
        name = name + '_layer6'
    if df.shape[1] == 1:
        df.columns = [name]
    df_all.append(df)
df_all = pd.concat(df_all, axis = 1)
df_all = df_all.loc[df_all.index.str.contains('OHDB'),:].reset_index().rename(columns={'index':'ohdb_id'})

# merge metadata
df_meta = pd.read_csv('../OHDB_metadata_subset.csv')
df_all = df_all.merge(df_meta, on = 'ohdb_id')

df_all.to_csv('../basin_attributes.csv', index = False)


### use GDAT reservoir area / catchment area to indicate the impact of reservoir regulation

In [ ]:
gdf_res = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_catchments.shp').to_crs('epsg:8857')
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp').to_crs('epsg:8857')
gdf_basin = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')

In [ ]:
inter = gpd.sjoin(gdf_basin, gdf_dam)
gdf_res['darea'] = gdf_res.area / 1000000
inter = inter.merge(gdf_res[['Feature_ID','Dam_Name','darea']], on = ['Feature_ID','Dam_Name'])
inter.loc[inter.Year_Fin=='BLANK','Year_Fin'] = None
inter['Year_Fin'] = pd.to_numeric(inter['Year_Fin'])

inter.loc[inter.Year_Const.isna(),'Year_Const'] = np.nan
inter.loc[~inter.Year_Const.isna(),'Year_Const'] = inter.loc[~inter.Year_Const.isna(),'Year_Const'].str[:4].astype(int)
inter['year'] = inter[['Year_Fin','Year_Const']].min(axis=1)
inter = inter.groupby(['ohdb_id','gritDarea']).apply(lambda x:pd.Series([
    x.darea.sum(),
    x.year.mean(),
    x.Main_P_Map.mode().values[0] if len(x.Main_P_Map.mode().values)>=1 else 'Hydroelectricity'
], index = ['res_darea_normalize','Year_ave','Main_Purpose_mode'])).reset_index()
inter['Year_ave'] = inter['Year_ave'].fillna(2000)
inter['Main_Purpose_mode'] = inter['Main_Purpose_mode'].fillna('Hydroelectricity')
inter['res_darea_normalize'] = inter.res_darea_normalize / inter.gritDarea
inter.to_csv('../data/dam_impact.csv', index = False)
print(inter.head())

AttributeError: 'DataFrame' object has no attribute 'Year_ave'

### add country ID

In [ ]:
import matplotlib.pyplot as plt
df = pd.read_csv('../data/basin_attributes.csv')
gdf_gauge = gpd.GeoDataFrame(data = df, geometry = gpd.points_from_xy(df.ohdb_longitude, df.ohdb_latitude), crs = 'epsg:4326').to_crs('epsg:8857')
gdf_country = read_dataframe('../geography/ne_10m_admin_0_countries_lakes.shp').to_crs('epsg:8857')
fig, ax = plt.subplots()
gdf_country.plot(ax = ax, column = 'SOVEREIGNT')
gdf_gauge.plot(ax = ax, zorder = 3, markersize = .1, color = 'r')

In [ ]:
df = pd.read_csv('../data/basin_attributes.csv')
df = df.loc[:,~df.columns.str.contains('country')]
df = df.loc[:,~df.columns.str.contains('SOVEREIGNT')]
inter = gpd.sjoin(gdf_gauge, gdf_country[['geometry','SOVEREIGNT']], how = 'left')
inter1  = inter.loc[~inter.SOVEREIGNT.isna(),:]
gdf_gauge2 = gdf_gauge.loc[~gdf_gauge.ohdb_id.isin(inter1.ohdb_id.values),:]
inter2 = []
for i in range(gdf_gauge2.shape[0]):
    tmp = gpd.sjoin_nearest(gdf_gauge2.iloc[[i],:], gdf_country[['geometry','SOVEREIGNT']], how = 'left')
    inter2.append(tmp)
inter2 = pd.concat(inter2)
inter = pd.concat([inter1, inter2])
print(inter.shape, inter.ohdb_id.unique().shape, inter.shape, gdf_gauge.shape)
df = df.merge(inter[['ohdb_id','SOVEREIGNT']], on = 'ohdb_id').rename(columns = {'SOVEREIGNT':'country'})

df.to_csv('../data/basin_attributes.csv', index = False)

### calculate annual average and std of Tmax and Tmin as basin attributes

In [ ]:
import pandas as pd
import glob
fnames = glob.glob('../data_mswx/*meteo*')
def func(fname):
    df = pd.read_csv(fname)
    df = df.set_index('ohdb_id')
    df = df.loc[:,(df.columns.str.contains('Tmax'))|(df.columns.str.contains('Tmin'))]
    return (df)
import multiprocessing as mp
pool = mp.Pool(8)
df = pool.map(func, fnames)
df = pd.concat(df, axis = 1)
df_Tmax = pd.concat([
    df.loc[:,df.columns.str.contains('Tmax')].mean(axis = 1),
    df.loc[:,df.columns.str.contains('Tmax')].std(axis = 1)
], axis = 1)
df_Tmin = pd.concat([
    df.loc[:,df.columns.str.contains('Tmin')].mean(axis = 1),
    df.loc[:,df.columns.str.contains('Tmin')].std(axis = 1)
], axis = 1)

df_Tmax.columns = ['tmax_ave','tmax_std']
df_Tmin.columns = ['tmin_ave','tmin_std']
df_Tmax.head()

In [ ]:
df_Tmax.columns = ['tmax_ave','tmax_std']
df_Tmin.columns = ['tmin_ave','tmin_std']
df_attr = pd.read_csv('../data/basin_attributes.csv').set_index('ohdb_id')
df_attr = df_attr.loc[:,~df_attr.columns.str.contains('tmax')]
df_attr = df_attr.loc[:,~df_attr.columns.str.contains('tmin')]
df_attr = pd.concat([df_Tmax, df_attr, df_Tmin], axis = 1).reset_index()
df_attr.to_csv('../data/basin_attributes.csv', index = False)

df_attr.head()

In [ ]:
!python merge_dataset_for_modeling.py Qmin7

In [ ]:
!ls ../data/*final*season*multi*MSWX* -l

In [ ]:
df = pd.read_csv('../data/Qmin7_final_dataset_seasonal4_multi_MSWX_meteo.csv')
df.loc[df.tmax_3<0,:].shape

In [ ]:
!time python tmp.py ../data/Qmin7_final_dataset_seasonal_multi_MSWX_meteo.csv rf